In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from datetime import datetime

In [5]:
plt.style.use('seaborn-v0_8-darkgrid')

In [7]:
oil_data = pd.read_csv('../data/BrentOilPrices.csv')

In [ ]:
print(oil_data.head())
print(oil_data.info())


        Date  Price
0  20-May-87  18.63
1  21-May-87  18.45
2  22-May-87  18.55
3  25-May-87  18.60
4  26-May-87  18.63
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9011 entries, 0 to 9010
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    9011 non-null   object 
 1   Price   9011 non-null   float64
dtypes: float64(1), object(1)
memory usage: 140.9+ KB
None


In [9]:
print(oil_data.dtypes)

Date      object
Price    float64
dtype: object


In [11]:
oil_data['Date'] = pd.to_datetime(oil_data['Date'], format='%m-%b-%y')

In [13]:
print(oil_data.dtypes)
print(oil_data.head())

Date     datetime64[ns]
Price           float64
dtype: object
        Date  Price
0 1987-05-20  18.63
1 1987-05-21  18.45
2 1987-05-22  18.55
3 1987-05-25  18.60
4 1987-05-26  18.63
